In [1]:
%pip install ultralytics


Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install kagglehub


Note: you may need to restart the kernel to use updated packages.


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("fatihkgg/affectnet-yolo-format")

print("Path to dataset files:", path)

100%|██████████| 258M/258M [00:11<00:00, 22.9MB/s] 

Extracting files...


Path to dataset files: /home/sagemaker-user/.cache/kagglehub/datasets/fatihkgg/affectnet-yolo-format/versions/2


In [6]:
%cp -r /home/sagemaker-user/.cache/kagglehub/datasets/fatihkgg/affectnet-yolo-format/versions/2/YOLO_format .

In [1]:
from ultralytics import YOLO

# ---------------- Hyper‑parameters -----------------
EPOCHS = 200
BATCH  = 16
IMGSZ  = 96
DEVICE = '0'   # GPU id   (use 'cpu' to force CPU)
# ---------------------------------------------------

model = YOLO('yolov8m.pt')   # base nano model
# model = YOLO("./runs/train/affectnet_yolov8_tuned2/weights/last.pt")


100%|██████████| 49.7M/49.7M [00:00<00:00, 54.4MB/s]


In [ ]:
results = model.train(
    # — DATA ----------------------------------------------------------
    data='./YOLO_format/data.yaml',      # same splits as before
    imgsz=IMGSZ,                  # upscale a bit (96→128) but stay GPU‑friendly
    batch=BATCH,                   # 16 GB T4 fits 64 @128px with v8‑l
    single_cls=False,           # keep 8 emotions separate
    cache="ram",                # tiny dataset ⇒ RAM‑cache accelerates

    # — OPTIMISER & SCHEDULER ----------------------------------------
    epochs=EPOCHS,                 # plenty for convergence; see patience
    optimizer="AdamW",          # good for classification‑heavy tasks
    lr0=0.005,                  # lower than v8 default (0.01) for large net
    lrf=0.01,                   # cosine final LR = lr0*0.01
    momentum=0.9,               # β1 if AdamW (`momentum` still applies)
    weight_decay=0.0005,    # mild L2 to preserve fine facial features
    cos_lr=True,            # cosine annealing schedule
    warmup_epochs=2,        # short warm‑up

    # — AUGMENTATION -------------------------------------------------
    hsv_h=0.01, hsv_s=0.6, hsv_v=0.3,  # moderate colour jitter
    flipud=0.0, fliplr=0.5,            # no vertical flips; keep horizontals
    scale=0.5,                         # random zoom ±50 %
    shear=0.0,                          # avoid geometric warps that distort faces
    close_mosaic=0,                    # disable mosaic for small crops
    mixup=0.0,                          # mixing two faces confuses labels
    dropout=0.20,                      # light dropout in head for generalisation

    # — TRAINING BEHAVIOUR -------------------------------------------
    device=DEVICE,                          # GPU id
    workers=8,                         # dataloader threads
    amp=True,                          # mixed‑precision for speed
    freeze=10,                         # lock first ≈10 layers (generic edges)
    resume=False,                      # set True if picking up from last.pt
    plots=True,
    project="runs/train",      
    name="affectnet_yolov8_med_tuned"
)



Ultralytics 8.3.112 🚀 Python-3.12.9 torch-2.5.1 CUDA:0 (Tesla T4, 14918MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=./YOLO_format/data.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=96, save=True, save_period=-1, cache=ram, device=0, workers=8, project=runs/train, name=affectnet_yolov8_med_tuned, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.2, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, li

train: Scanning /home/sagemaker-user/YOLO_format/train/labels.cache... 17101 images, 0 backgrounds, 0 corrupt: 100%|██████████| 17101/17101 [00:00<?, ?it/s]

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.



train: Caching images (0.4GB RAM): 100%|██████████| 17101/17101 [00:08<00:00, 1933.42it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 344.3±261.6 MB/s, size: 9.1 KB)


...

    107/200     0.781G        inf     0.3378     0.8868         45         96: 100%|██████████| 1069/1069 [01:00<00:00, 17.69it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 169/169 [00:15<00:00, 10.89it/s]


                   all       5406       5406      0.531      0.663      0.629      0.535

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    108/200     0.781G     0.0666     0.3368     0.8878         38         96: 100%|██████████| 1069/1069 [00:58<00:00, 18.40it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 169/169 [00:15<00:00, 10.98it/s]


                   all       5406       5406      0.535       0.66       0.63      0.536

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    109/200     0.781G    0.06602     0.3346     0.8855         52         96:  15%|█▍        | 157/1069 [00:08<00:49, 18.54it/s]